In [1]:
!pip install pycuda ThrustRTC scikit-cuda

     |████████████████████████████████| 1.7 MB 9.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 765 kB 38.8 MB/s 
     |████████████████████████████████| 114 kB 35.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627345 sha256=a92c2e602504431f9edcba060baf44daa2c70925a6eec24f1c6861339bfd9e39
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.8-py2.py3-none-any.whl size=60725 sha256=984e8591d0ff6773550483b7a452dd5e81e60cc4e74c501b3eaece9aa2335f05
  Stored in directory: /root/.cache/pip/wheels/bd/2d/ef/0127a17bafa44971f11d05d0e38d7947144cf9e33313bf12a7
Successfully built pycuda pytools


In [83]:
!pip install fsspec

     |████████████████████████████████| 125 kB 9.5 MB/s 


In [ ]:
import jax.numpy as jnp

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import sys
import os
path = "/content/drive/My Drive"
sys.path.append(path)
os.chdir(path)
%cd graduation\ project/

/content/drive/My Drive/graduation project


In [3]:
!nvidia-smi

Mon Oct 11 04:01:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import numpy as np
import pandas as pd
from skcuda import cublas
from pycuda import gpuarray
import pycuda.autoinit
import time
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
import ThrustRTC as trtc
import numpy as np
import time
from matplotlib import pyplot
import pandas as pd
from sklearn import metrics
from collections import Counter
import random
import jax.numpy as jnp

FLOAT_MAX = 1e10

/usr/local/lib/python3.7/dist-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [5]:
# 求平方
get_square = SourceModule(
'''
__global__ void x_square(float *x, int &n)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx<n)
        x[idx] = x[idx] * x[idx];
}

__global__ void y_square(float *y, int &m)
{
    const int idx = threadIdx.x;
    if(idx<m)
        y[idx] = y[idx] * y[idx];

}
'''
)

# 求和
square_sum = SourceModule(
'''

__global__ void sum_row_xy(float *s_x, float *s_y, float *self_sum1, float *self_sum2, int &numSample, int &k, int &dim)
{
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    if(idx < k)
        for(int i=0;i<dim;i++)
            self_sum2[idx] += s_y[idx*dim+i];


    if(idx < numSample + k && idx >= k)    
        for(int j=0;j<dim;j++)    
            self_sum1[idx - k] += s_x[(idx - k)*dim+j];    
    
    // __syncthreads(); 
}


__global__ void sum_total(float *self_sum1, float *self_sum2, float *result, int &k, int &numSample)
{
    const int idx1 = threadIdx.x + blockIdx.x * blockDim.x;
    int quo = idx1 % numSample;
    int rem = idx1 / numSample;
    
    if(idx1 < numSample * k)
    
        result[idx1] = self_sum1[quo] + self_sum2[rem];

    __syncthreads();     

}


__global__ void sum_row_x(float *s_x, float *self_sum1, int &numSample, int &dim){
    const int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < numSample)
        for(int j=0;j<dim;j++)    
            self_sum1[idx] += s_x[idx*dim+j]; 
    __syncthreads();
}

__global__ void sum_row_x_reduction(float *s_x, float *self_sum1, int &numSample, int &dim){
    const int idx = blockIdx.x;
    int tid = threadIdx.x;  // 42 * 8
    __shared__ float s_xdata[1024];
    s_xdata[tid] = s_x[tid + dim*idx];
    __syncthreads();
    for(int i=16; i>0; i >>= 1){
      if(tid < i) s_xdata[tid] += s_xdata[tid+i];
      __syncthreads();
    }
    // if (tid < 32) warpReduce(s_xdata, tid);
    if(tid == 0){
      self_sum1[idx] = s_xdata[0]; 
      for(int j=32; j < dim; j++)
        self_sum1[idx] += s_xdata[j]; 
    }
}


__device__ void warpReduce(volatile float* sdata,int tid) {
    sdata[tid] += sdata[tid + 32];
    sdata[tid] += sdata[tid + 16];
    sdata[tid] += sdata[tid + 8];
    sdata[tid] += sdata[tid + 4];
    sdata[tid] += sdata[tid + 2];
    sdata[tid] += sdata[tid + 1];
}

__global__ void sum_row_y_reduction(float *s_y, float *self_sum2, int &dim){
    const int idx = blockIdx.x;
    int tid = threadIdx.x;
    __shared__ float s_ydata[1024];
    s_ydata[tid] = s_y[tid + dim*idx];
    __syncthreads();
    for(int i=16; i>0; i >>= 1){
      if(tid < i) s_ydata[tid] += s_ydata[tid+i];
      __syncthreads();
    }
    // if (tid < 16) warpReduce(s_ydata, tid);
    if(tid == 0){
      self_sum2[idx] = s_ydata[0]; 
      for(int j=32; j < dim; j++)
        self_sum2[idx] += s_ydata[j]; 
    }
    
    
}

'''
)


# shared_memory.max()=48KB(12000 int/float32)
find = SourceModule(
'''
__global__ void find_range(float *cluster, float *result)
{
    __shared__ float sdata[512];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x*blockDim.x+ threadIdx.x;
    sdata[tid] = cluster[i];
    if(sdata[tid] != sdata[tid+1] and tid < 499) // 统计各簇中元素的个数
        result[int(sdata[tid])] = i+1;
    __syncthreads();
}

'''
)

find_minimum = SourceModule(
'''
__global__ void find_min(float *list, float *result_value, int *result_idx, int &length, int &n)
{
    __shared__ float sdata[1024];
    __shared__ int ssdata[1024];
    unsigned int tid = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;   


    if(tid < length-1)
    {
        sdata[tid] = list[i + tid * length]; // store data
        ssdata[tid] = tid; // store index
    }
    sdata[length - 1] = 10000;
    ssdata[length - 1] = 23;  

    __syncthreads();
    
    unsigned int s;
    if(tid < length)
    {
        for(s = (blockDim.x+1)/2; s>0; s>>=1) {
            if (tid < s) 
            {
                ssdata[tid] = (sdata[tid] <= sdata[tid + s])? ssdata[tid]:ssdata[tid+s];
                __syncthreads();
                sdata[tid] = (sdata[tid] <= sdata[tid + s])? sdata[tid]:sdata[tid+s];
                __syncthreads();
            }

            __syncthreads();

}
        if(s == 0)
        {
            ssdata[0] = (sdata[0] <= sdata[n-1])? ssdata[0]:ssdata[n-1];
            sdata[0] = (sdata[0] <= sdata[n-1])? sdata[0]:sdata[n-1];
            __syncthreads();
        }    
    }
       
    if(tid == 0)
    {
        result_idx[blockIdx.x] = ssdata[0];
        result_value[blockIdx.x] = sdata[0];     
    }
}
'''
)


def cluster_assignment_new(C, k, numSample=494021):
    lengthC = k * numSample
    n = k    
    divider = k + 1 if(k % 2) else k        
    while(divider % 2 == 0):
        n = divider / 2
        divider /= 2
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)
    
    length = np.int32(k+1)
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result_value = gpuarray.to_gpu(np.zeros((lengthC // k, ), dtype = np.float32))
    result_idx = gpuarray.to_gpu(np.zeros((lengthC // k, ), dtype = np.int32))
    # C = gpuarray.to_gpu(C)

    find_func_hybrid = find_minimum.get_function("find_min")
    find_func_hybrid(C, result_value, result_idx, length_gpu, n_gpu, block=(k, 1, 1),grid=(lengthC // k, 1))
    cluster_assignment = np.zeros((len(result_idx),2))
    
    cluster_assignment[:,0] = result_idx.get()
    cluster_assignment[:,1] = result_value.get()
    return cluster_assignment

In [6]:
## test
s_y = np.ones((23, 42)).astype(np.float32).flatten()
s_y[0] = 2
s_y[41] = 4
s_y[42] = 5
s_y[-1] = 3
s_y = gpuarray.to_gpu(s_y)
sum2 = np.zeros((23, )).astype(np.float32)
sum2 = gpuarray.to_gpu(sum2)
k = np.int32(23)
k_gpu = cuda.mem_alloc(k.nbytes)
cuda.memcpy_htod(k_gpu, k)
dim = np.int32(42)
dim_gpu = cuda.mem_alloc(dim.nbytes)
cuda.memcpy_htod(dim_gpu, dim)
test = square_sum.get_function("sum_row_y_reduction")
# test = square_sum.get_function("sum_row_x")
test(s_y, sum2, dim_gpu, block=(42,1,1), grid=(23,1,1))
# test(s_y, sum2, k_gpu, dim_gpu, block=(1024,1,1), grid=(1,1,1))
sum2

array([46., 46., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42., 42.,
       42., 42., 42., 42., 42., 42., 42., 42., 42., 44.], dtype=float32)

In [1]:
# matrix1 is dataset, matrix2 is centroids
def get_x_square(matrix1, numSample=494021, dim=42):
    """
      It can be interpolated that the dataset square need to calculate for only once!
    """
    n = numSample * dim
    grid_dim = int(np.ceil(n / 1024))
    n = np.int32(n)
    n_gpu = cuda.mem_alloc(n.nbytes)
    cuda.memcpy_htod(n_gpu, n)

    x = matrix1.flatten().astype(np.float32)
    
    x_gpu = gpuarray.to_gpu(x)
    x_func = get_square.get_function("x_square")
    
    # we have a flexible block assignment
    x_func(x_gpu, n_gpu, block=(1024,1,1), grid=(grid_dim,1))
    return x_gpu # could be used as matrix1 in get_xy_square

def get_xy_square(x_gpu, matrix2, cluster_k, numSample, dim):
    m = cluster_k * dim
    # determining grid size
    m = np.int32(m)
    m_gpu = cuda.mem_alloc(m.nbytes)
    cuda.memcpy_htod(m_gpu, m)
    grid_dim = int(np.ceil(m / 1024))

    y_gpu = matrix2.astype(np.float32)

    # y_gpu = gpuarray.to_gpu(y)
    y_func = get_square.get_function("y_square")
    
    y_func(y_gpu, m_gpu, block=(1024,1,1), grid=(grid_dim,1))

    self_sum1 = np.zeros((numSample, ), dtype = np.float32)
    self_sum2 = np.zeros((cluster_k, ), dtype = np.float32)
    
    self_sum1_gpu = gpuarray.to_gpu(self_sum1)
    self_sum2_gpu = gpuarray.to_gpu(self_sum2)
    
    numSample_np = np.int32(numSample)
    numSample_gpu = cuda.mem_alloc(numSample_np.nbytes)
    cuda.memcpy_htod(numSample_gpu, numSample_np)
    
    k = np.int32(cluster_k)
    k_gpu = cuda.mem_alloc(k.nbytes)
    cuda.memcpy_htod(k_gpu, k)
    
    dim_ = np.int32(dim)
    dim_gpu = cuda.mem_alloc(dim_.nbytes)
    cuda.memcpy_htod(dim_gpu, dim_)

    result = np.zeros((cluster_k*numSample, ), dtype = np.float32)
    block_dim_result = len(result) // 1024 + 1
    
    result_gpu = gpuarray.to_gpu(result)
   
    # optimization v1.0
    # sum_func_row = square_sum.get_function('sum_row_xy')

    grid_dim_new = int(np.ceil(numSample / 1024))
    # sum_func_row(x_gpu, y_gpu, self_sum1_gpu, self_sum2_gpu, numSample_gpu, k_gpu, dim_gpu, block = (1024,1,1), grid = (grid_dim_new,1))
    sum_func_y = square_sum.get_function('sum_row_y_reduction')
    sum_func_y(y_gpu, self_sum2_gpu, dim_gpu, block=(dim, 1, 1), grid=(cluster_k, 1, 1))

    sum_func_x = square_sum.get_function('sum_row_x')
    sum_func_x(x_gpu, self_sum1_gpu, numSample_gpu, dim_gpu, block=(1024, 1, 1), grid=(grid_dim_new, 1, 1))

    sum_func_total = square_sum.get_function("sum_total")
    sum_func_total(self_sum1_gpu, self_sum2_gpu, result_gpu, k_gpu, numSample_gpu, block=(1024,1,1), grid=(block_dim_result,1))
    
    return result_gpu # result_gpu could be used for matrix3 in dis_computation


# 计算与质心之间的距离
def dis_computation(matrix1, matrix2, matrix3, orishape_matrix1):
    """matrix 1 is centroids, 
    matrix 2 is dataset.T, 
    matrix 3 is result"""
    matrix1 = matrix1.astype(np.float32)
    matrix2 = matrix2.astype(np.float32)
    
    m = orishape_matrix1[0]
    n = matrix2.shape[1]
    
    k = orishape_matrix1[1]
    B_gpu = matrix1
    #the above is matrix B

    A_gpu = matrix2
    # the above is matrix A
    transa = 'n'
    transb = 'n'
    alpha = -2
    beta = 1
    # computing matrix
    h = cublas.cublasCreate()
    cublas.cublasSgemm(h, transa, transb, n, m, k, alpha, A_gpu.gpudata, n, B_gpu.gpudata, k, beta, matrix3.gpudata, n)
    cublas.cublasDestroy(h)
    
    C = matrix3.get()
    return C


def cluster_assignment(C, k=23, numSample=494021):
    if(len(C) == 1):
        return np.array([C, ])
    C = C.reshape((k, numSample))
    # minDist = np.amin(C, axis = 0)
    minDistIdx = np.argmin(C, axis = 0)
    # cluster_assignment = np.vstack((minDistIdx, minDist)).T
    return minDistIdx



def sort_by_key_without_count(clusterAssment):
    cluster = clusterAssment[:,0].astype(np.float32)
    cluster_gpu = trtc.device_vector_from_numpy(cluster)
    n = cluster_gpu.size()
    index = trtc.device_vector("int32_t", n)
    trtc.Sequence(index)
    trtc.Sort_By_Key(cluster_gpu, index)
    index_cpu = index.to_host()
    cluster_cpu = cluster_gpu.to_host()
    return index_cpu, cluster_cpu


def sort_gpu(index_cpu, cluster_cpu, dataset, k, placement):
    cluster1 = cluster_cpu[0:250000].astype(np.float32)
    cluster2 = cluster_cpu[250000:-1].astype(np.float32)
    result1 = np.zeros((k-1,),dtype=np.float32)
    result2 = np.zeros((k-1,),dtype=np.float32)
    find_func = find.get_function("find_range")
    find_func(cuda.In(cluster1),cuda.Out(result1), block=(500,1,1),grid=(500,1))
    find_func(cuda.In(cluster2),cuda.Out(result2), block=(500,1,1),grid=(500,1))
    # 取置信值，这里的代码可以优化一下
    # 如果数据量控制在25w以内效果最佳
    for j in range(len(result2)):
        if(result2[j] != 0):
            result2[j] += 250000 # 分批处理，第一批计数从0开始，第二批从250000开始
    for i in range(len(result2)-1):
        if(result2[i] > result2[i+1]): # 消除异常值
            result2[i] = 0
    starting_points = [0]
    for jj in range(len(result2)):
        starting_points.append(max(result1[jj], result2[jj]))
    starting_points.append(dataset.shape[0]+1)
    data_in_cluster = []
    start = int(starting_points[placement])
    end = int(starting_points[placement+1])
    data_in_cluster = dataset[index_cpu[start:end],:]
    data_in_cluster = np.array(data_in_cluster)
    return data_in_cluster



def cluster_assignment_gpu(C):
    C = C.astype(np.float32)
    length = np.int32(C.shape[1])
    length_gpu = cuda.mem_alloc(length.nbytes)
    cuda.memcpy_htod(length_gpu, length)
    result = np.zeros((C.shape[0],), dtype = np.float32)
    # 找到距离最近的质心点
    find_func = find.get_function("find_minimum")
    find_func(cuda.In(C),cuda.Out(result),length_gpu,block=(C.shape[1],1,1),grid=(C.shape[0],1))
    
    result_idx = np.argmin(C, axis = 1)    
    cluster_assignment = np.zeros((len(result_idx),2))
    cluster_assignment[:,0] = result_idx
    cluster_assignment[:,1] = result
    return cluster_assignment


In [8]:
import pycuda


# cpu的版本
def euclDistance(vector1,vector2):
    return np.sqrt(sum(np.power(vector2-vector1, 2))) #power计算次方

##初始化数据的中心点，k表示聚类中心数
##随机生成k个聚类中心
def initCentroids(dataset, k):
    numSample,dim = dataset.shape
    centroids = np.zeros((k,dim))
    for i in range(1,k+1):
        #index=int(np.random.uniform(0,numSample))#随机生成数
        index = int(i*10000)
        centroids[i-1,:] = dataset[index,:]
    return centroids

##kmean算法
def kmeans(dataset, k):
    pycuda.tools.clear_context_caches()
    numSample, dim = dataset.shape
    #生成新的两列数组，保存聚类信息
    # 第一列表示所属聚类中心，第二列表示与中心的误差
    clusterAssment = np.zeros((numSample, ))#这里dtype就默认
    clusterChanged = True

## step1 初始化聚类中心
    centroids = initCentroids(dataset, k)
    buffer = np.empty((numSample, ))
    itr = 0
    dataset_gpu = gpuarray.to_gpu(dataset.T)
    dataset_gpu_square = get_x_square(dataset)
    orishape_matrix1 = centroids.shape

    while itr < 28:
        itr += 1
        count = 0
        comparable_list = zip(buffer, clusterAssment)

        for val1, val2 in comparable_list:           
            if val1 != val2:
                count += 1
                if(count > numSample // 10000):
                    clusterChanged = True
                    break
            else:
                clusterChanged = False
        if clusterChanged is False:
            break
        
        buffer = clusterAssment[:]
        #二重循环：对所有数据点，与k个聚类中心计算距离
        #并保存标签与距离
        matrix1 = gpuarray.to_gpu(centroids.flatten())
        matrix2 = dataset_gpu
        matrix3 = get_xy_square(dataset_gpu_square, matrix1, k, numSample, dim)
        distance_mat = dis_computation(matrix1, matrix2, matrix3, orishape_matrix1)  # matrix1: dataset, matrix2: centroids.T, matrix3: (dataset.shape[0], centroids.shape[0]) 
        clusterAssment = cluster_assignment(distance_mat, k)
## step4 循环结束后更新聚类中心
        
        for i in range(k):
            comp = np.nonzero(clusterAssment == i)[0] # 当前状态的聚类情况
            comp_buffer = np.nonzero(buffer == i)[0] # 上一状态的聚类情况
            if(np.array_equal(comp, comp_buffer)):
                 # 当前簇内元素没有改变？如果没有改变，下次聚类忽略该簇
                continue
            pointsInCluster = dataset[comp]
            centroids[i, :] = np.mean(pointsInCluster, axis=0)
    ##循环结束，返回聚类中心和标签信息
    return centroids, clusterAssment

In [12]:
import dask.dataframe as dd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 


def evaluation(category_pre, category):
    real = Counter(category)
    pre = Counter(category_pre)
    print("Real category is ", real.items())
    print("Predicted clustering is ", pre.items())
    real = real.most_common()
    pre = pre.most_common()
    for j in range(numSample):
        for nn in range(k):
            if(category[j] == real[nn][0]):
                category[j] = int(pre[nn][0])
    ARI = metrics.adjusted_rand_score(category, category_pre)
    AMI = metrics.adjusted_mutual_info_score(category, category_pre)
    print("The ARI value is {:.4f}".format(ARI))
    print("The AMI value is {:.4f}".format(AMI))


if __name__ == "__main__":
    ## load data
    dataset = dd.read_csv('kdd_pre_final.csv',sep=',')
    # 真实的标签
    category_real = dataset.loc[:,["classification"]]
 
    dataset = dataset.iloc[:,:-1] 
    dataset = np.array(dataset)
    numSample = dataset.shape[0]
    ##  k表示聚类中心数
    k = 23
    total_time = 0
    times = 5
    start = time.time()
    for _ in range(times):
        centroids, clusterAssment = kmeans(dataset, k)
    end = time.time()
    print('algorithm (for training) average time: %.2f seconds'%((end - start) / times))
    
    category_real = np.array(category_real)
    category = np.array(list(map(lambda x: x[0], category_real)))
    category_pre = np.array(clusterAssment, dtype = np.int8)
    evaluation(category_pre, category.astype(np.int8))


algorithm (for training) average time: 5.58 seconds
Real category is  dict_items([(0, 97278), (1, 30), (2, 9), (3, 3), (4, 107201), (5, 280790), (6, 53), (7, 264), (8, 979), (9, 1040), (10, 1247), (11, 21), (12, 8), (13, 2203), (14, 12), (15, 1589), (16, 4), (17, 231), (18, 7), (19, 20), (20, 1020), (21, 2), (22, 10)])
Predicted clustering is  dict_items([(13, 37371), (1, 5816), (8, 38218), (2, 10479), (6, 43), (7, 5393), (5, 38215), (19, 56), (22, 59), (4, 253494), (21, 7861), (18, 11683), (14, 294), (9, 5704), (17, 1598), (16, 53), (12, 1243), (0, 2316), (3, 5098), (11, 48529), (15, 24), (20, 17), (10, 20457)])
The ARI value is 0.3577
The AMI value is 0.4884


In [ ]:
d = dict()
for i in range(k):
    d[i] = np.nonzero(category_pre == i)[0] # 当前状态的聚类情况
tsne = TSNE(n_components=2) 
dataset_tsne = tsne.fit_transform(dataset) 
vis_x, vis_y = dataset_tsne[:,0], dataset_tsne[:,1]
for key in d:
    plt.scatter(vis_x[d[key]], vis_y[d[key]], cmap=plt.cm.get_cmap("jet", k))
plt.show()